### Test URLS
https://sunstonepartners.com \
https://www.appliedlearning.com \
https://www.forsalebyowner.com login \
https://ecmins.com/ \
http://www.iconnect-corp.com \
https://cessco.ca/ ROBOT \
http://www.ticss.net \
https://www.tyremarket.com/Car-Tyres \
https://www.dentalxchange.com/ \

**TODOList**:

-Improve URL relevance check (exclude /#, /login, /sign-up)

-Never return empty nav object, instead string. If after try bs4 and sel, fails

-script defers to selenium if bs4 does nav_scrape but nav still empty (ex: https://www.forsalebyowner.com)

-File output with all columns

-Enhance href relevance function (both contain base_url)

-Four columns of information for each website

-Improve speed of sel nav_tree recursion

-retry if page_result is empty after page scrape

-page scrape for bs4 (page scrape working for sel)

-assess whitespace split to help headers

-requests 200 requirement for first href selection

-account for more options in 'assess' functions

-add website_url parameter into sel_nav_scrape for consistency

-FIX nav scrape for sel (nav scrape working for bs4). Specifically, first_href - not critical because very slow.

> Figure out alternative when no nav (all a tags' hrefs or first relevant href?) (ex: https://www.forsalebyowner.com) \
> Make sure that first_href returned is a URL in both bs4_nav_scrape() and sel_nav_scrape() \

-first text on page (home/about)

-mistral given as many pages as possible (via nav) -> really slow (~15 mins) save until next step

-utilize irrelevant in asses_href


### All imports

In [2]:
import requests
import pandas as pd
import validators
import time
from openai import OpenAI
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium_stealth import stealth
from selenium import webdriver
import re
import urllib.parse as up
from bs4 import BeautifulSoup

import asyncio
from urllib.parse import urlparse
import aiohttp
import nest_asyncio

### Important meta tags

In [11]:
def get_meta_tags(url): #: str) -> dict[str,str]:

    api_key = 'pk_d22cfd76694dc6566ccbf7ed6f50cdc66215091c'    

#     url = 'https://cessco.ca/'
    # url = 'https://www.appliedlearning.com'

    params = {'url': url, 'api_key': api_key}
    
    # Free JsonLink limit is 30 req/minute so wait 3 seconds just in case
    time.sleep(3)
    
    response = requests.get('https://jsonlink.io/api/extract', params=params)

    if response.status_code == 200:
        data = response.json()
#         print(data)
        print('Title: ', data['title'], '\nDescription: ', data['description'], '\nDomain: ', data['domain'])
        return {'metadata':{k: data.get(k,None) for k in ('title', 'description', 'domain')}}
    else:
        print(f'JSONLink Error for {url}: {response.status_code} - {response.text}')
        return {'metadata': 'Metadata unavailable'}

In [ ]:
get_meta_tags('https://www.dentalxchange.com/')

### Handle navigation

#### Get all a tags (get best nav)
1. test if href valid url \
2. test if url + (optional /) + href valid url \
3. likely a bust

####  Selenium

In [3]:
def sel_normalize_whitespace(text):
    # Replace one or more whitespace characters (including spaces, tabs, and newlines) with a single space
    return re.sub(r'\s+', ' ', text).strip()

def sel_assess_href(base_url: str, href: str) -> str:
    if not validators.url(href):
        href = up.urljoin(base_url,href)
    # Add functionality here to compare if one is contained in the other
    return [href, 'relevant' if up.urlparse(href).netloc == up.urlparse(base_url).netloc else 'irrelevant']

def sel_find_relevant_hrefs(driver):
    atags = driver.find_elements("xpath","//a")
    relevant_hrefs = []
    for a in atags:
        text, href = sel_normalize_whitespace(a.get_attribute('textContent')),a.get_attribute('href')
        website_url = driver.current_url
        assessed_href = sel_assess_href(website_url, href)
        if assessed_href[1] == 'relevant' and (assessed_href[0] != website_url and text != 'Skip to content'):
            relevant_hrefs += [(text, assessed_href[0])]
    return relevant_hrefs

def sel_find_first_href(nested_list) -> str:
    for item in nested_list:    
        if isinstance(item, list):
                    # Recursively search within the list
                    result = sel_find_first_href(item)
                    if result:  # If a valid URL is found in the recursion, return it
                        return result
        elif isinstance(item, tuple) and len(item) == 2:
            # If the item is a tuple with 2 elements, check the second element for a valid URL
            if validators.url(item[1]):
                return item[1]  # Return the URL if it's valid
    return 'No href found'

def sel_build_tree(base_url,element):
    # Initialize the node with tag name and text content
    node_contents = {'text': sel_normalize_whitespace(element.get_attribute('textContent')),
                     'href': sel_assess_href(base_url,element.get_attribute('href'))[0]} if element.tag_name == 'a' else {}
    node = {
        **node_contents,
        'children': []
    }

    # Recursively build the tree for each child element
    children = element.find_elements(By.XPATH, "./*")  # Only direct children
    for child in children:
        node['children'].append(sel_build_tree(base_url,child))

    if not node['children'] or all(not obj for obj in node['children']):
        del node['children']

    return node

def sel_convert_tree(root) -> list[list,int]:
    ans, total_hrefs = [], 0
    if 'children' not in root:
        if 'text' in root and 'href' in root:
            return (root['text'], root['href'], 1)
    else:
        for child in root['children']:
            links = sel_convert_tree(child)
            if links:
                total_hrefs += links[-1]
                ans += [links[:-1]]

    return [*list(filter(None, ans)), total_hrefs]

# TODO: Fix this - first_href must be a URL (even if no nav, find first relevant href but need to broaden def of relevant)
# Returns the nav tree (either advanced nested or basic list of hrefs) and first href
def sel_nav_scrape(driver) -> list[list[tuple],str]:
    sel_nav_return, nav_trees = [], []

    # Find navs, construct trees and find max
    navs = driver.find_elements("xpath","//nav")
    for nav in navs:
        nav_trees.append(sel_build_tree(driver.current_url, nav))
    max_nav = ({}, 0)
    for tree in nav_trees:
        converted = sel_convert_tree(tree)
        if converted[-1] > max_nav[-1]:
            max_nav = converted
    # [:-1] to account for nested tree
    max_nav_tree = max_nav[:-1]

    # Construct return
    if not max_nav[0]:
    # if max_nav == ({}, 0): #or len(max_nav[-1]) < x:
        # If no/not enough navs, find all relevant atags
        relevant_hrefs = sel_find_relevant_hrefs(driver)  
        sel_nav_return.append(relevant_hrefs)
        first_href = relevant_hrefs[0][1] if relevant_hrefs else 'No href found'
    else:
        sel_nav_return.append(max_nav_tree)
        first_href = sel_find_first_href(max_nav)

    sel_nav_return.append(first_href)
    return sel_nav_return

driver = webdriver.Chrome()
driver.get('https://forsalebyowner.com/')
sel_nav = sel_nav_scrape(driver)
driver.close()
sel_nav

[[('Sell Your Home', 'https://www.forsalebyowner.com/sellyourhome/package'),
  ("What's My Home Worth",
   'https://www.forsalebyowner.com/what-is-my-home-worth'),
  ("Seller's Guide",
   'https://www.forsalebyowner.com/seller-guide/should-you-sell-your-house'),
  ('Closing Assistance',
   'https://www.forsalebyowner.com/sellyourhome/closing-assistance'),
  ('Search For Homes', 'https://www.forsalebyowner.com/homes-for-sale'),
  ('Learning Center', 'https://www.forsalebyowner.com/blog'),
  ('Frequently Asked Questions',
   'https://www.forsalebyowner.com/support/faq/forsalebyowner-basics'),
  ('USA Property Directory', 'https://www.forsalebyowner.com/property'),
  ('Create My Listing', 'https://www.forsalebyowner.com/sellyourhome/package'),
  ('Whitney, TX130Active Listings1New Listing',
   'https://www.forsalebyowner.com/search/list/whitney-texas/fsbo-source/for_sale-status'),
  ('Granbury, TX69Active Listings1New Listing',
   'https://www.forsalebyowner.com/search/list/granbury-texas

#### bs4

In [4]:
def bs4_build_tree(base_url, element):
    # Initialize the node with tag name and text content
    node_text = {'text': element.get_text(strip=True)} if element.name == 'a' else {}
    node = {
        **node_text,
        'children': []
    }

    # If it's an <a> tag, include the href attribute
    if element.name == 'a':
        node['href'] = bs4_assess_href(base_url,element.get('href'))[0]

    # Recursively build the tree for each child element
    for child in element.find_all(recursive=False):  # Only direct children
        node['children'].append(bs4_build_tree(base_url, child))
        
    if not node['children'] or all(not obj for obj in node['children']):
        del node['children']

    return node

def bs4_convert_tree(root):
    ans, total_hrefs = [], 0
    if 'children' not in root:
        if 'text' in root and 'href' in root:
            return (root['text'],'' if root['href'] == 'javascript:void(0);' else root['href'],1)
            # Aesthetic output
            # return (f"root['text']}-> {root['href']}",1)
        # else:
        #     return ['','',0]
    else:
        for child in root['children']:
            # print(child)
            links = bs4_convert_tree(child)
            if links:
                # print(links)
                total_hrefs += links[-1]
                ans += [links[:-1]]
            
    return [*list(filter(None,ans)),total_hrefs]

# Need to handle javascript:void(0); case
def bs4_assess_href(base_url, href) -> str:
    if not validators.url(href):
        href = up.urljoin(base_url,href)
    return [href, 'relevant' if up.urlparse(href).netloc == up.urlparse(base_url).netloc else 'irrelevant']

def bs4_find_relevant_hrefs(soup, website_url: str) -> list[tuple[str, str]]:
    atags = soup.find_all('a')
    relevant_hrefs = []
    for a in atags:
        text, href = a.get_text(strip=True),a.get('href')
        assessed_href = bs4_assess_href(website_url, href)
        if assessed_href[1] == 'relevant' and (assessed_href[0] != website_url and text != 'Skip to content'):
            relevant_hrefs += [(text, assessed_href[0])]
    return relevant_hrefs

def bs4_find_first_href(nested_list) -> str:
    for item in nested_list:    
        if isinstance(item, list):
                    # Recursively search within the list
                    result = bs4_find_first_href(item)
                    if result:  # If a valid URL is found in the recursion, return it
                        return result
        elif isinstance(item, tuple) and len(item) == 2:
            if validators.url(item[1]):
                return item[1]  # Return the URL if it's valid
    return 'No href found'

def bs4_nav_scrape(website_url: str, soup) -> list[tuple[str,str],str]:
    bs4_nav_return, nav_trees = [], []

    # Find navs, construct trees, find max
    navs = soup.find_all('nav')
    for nav in navs:
        nav_trees.append(bs4_build_tree(website_url, nav))
    max_nav = ({},0)
    for tree in nav_trees:
        converted = bs4_convert_tree(tree)
        if converted[-1] > max_nav[-1]:
            max_nav = converted
    # [:-1] to account for nested tree
    bs4_max_nav_tree = max_nav[:-1]

    # Construct return
    if not max_nav[0]: #or len(max_nav[-1]) < x:
        # If no/not enough navs, find all relevant atags
        relevant_hrefs = bs4_find_relevant_hrefs(soup, website_url)
        bs4_nav_return.append(relevant_hrefs)
        first_href = relevant_hrefs[0][1] if relevant_hrefs else ''
    else:
        bs4_nav_return.append(bs4_max_nav_tree)
        first_href = bs4_find_first_href(max_nav)

    bs4_nav_return.append(first_href)
    return bs4_nav_return

# url = 'https://www.dentalxchange.com/'
# url = 'https://ecmins.com/'
# url = 'https://iquartic.com/' # blocked on requests
# url = 'https://www.ripoffreportremovalhelp.com/' # blocked on requests
# url = 'https://pulseca.com/'
url_test = 'https://www.scorpion.co/'

html = requests.get(url_test).content
soupt = BeautifulSoup(html, 'html.parser')

url_test = 'https://www.pavestone.com/'
response = requests.get(url_test)
soupy = BeautifulSoup(response.text, 'html.parser')

# response = requests.get(url_test).content
# soupr = BeautifulSoup(response, 'html.parser')
bs4_nav_scrape(url_test, soupy)
# soupy.find_all('a')
# print(soupr.find('h2'))


[[('Product Search A-Z', 'https://www.pavestone.com/a-z'),
  ('Where To Buy', 'https://www.pavestone.com/where-to-buy'),
  ('Pavestone for the Pro', 'https://www.pavestone.com/contractors'),
  ('Technical Resources', 'https://www.pavestone.com/resources/'),
  ('Project Profiles', 'https://www.pavestone.com/project-profiles/'),
  ('Advanced Product Search', 'https://www.pavestone.com/productsearch'),
  ('Pavers', 'https://www.pavestone.com/productcategory/pavers'),
  ('Retaining Walls',
   'https://www.pavestone.com/productcategory/retaining-walls'),
  ('Patio Stones', 'https://www.pavestone.com/productcategory/stepstones'),
  ('Edging', 'https://www.pavestone.com/productcategory/edgers'),
  ('RumbleStone', 'https://www.pavestone.com/productfamily/rumblestone'),
  ('Fire Pit Kits', 'https://www.pavestone.com/projectfamily/Fire Pits'),
  ('Outdoor Living',
   'https://www.pavestone.com/projectcategory/outdoor-living'),
  ('Project Gallery', 'https://www.pavestone.com/gallery'),
  ('Resou

**Report:** it seems as though the javascript:void(0); case is handled because validators.url thinks it's valid, but the netloc's are not the same, so it's labelled irrelevant. \
**TODO:** Need to figure out nav name (the text only in the nav element, not in the contained a's, create tree-like structure. 

### Scraping Methods

In [5]:
def word_count(seg):
    count = 0
    for i in seg:
        if i == ' ':
            count += 1
    return count+1

#### bs4

In [16]:
def bs4_pages_scrape(urls: list[str]) -> list[dict]:
    pages = []
    for url in urls:
        if url and validators.url(url):
            try:
                response = requests.get(url).text
            except Exception as e:
                print(f'HTTPRequest error: {e}')
                pages.append({'headers':['Page not available']})
                return pages
            soup = BeautifulSoup(response, 'html.parser')
            # Split on any whitespace (\n and \t) -> maybe this is causing weird headers
            page_text = soup.get_text("|",strip=True).split("|")
            # Extract the first two pieces of text with more than (7) words -> to be tested
            first_relevant = {'first_relevant': [i for i in page_text if word_count(i) > 7][:2]}
            # Two longest pieces of text on the page. Test if this produces relevant results
            two_longest = {'two_longest': sorted(page_text,key=len)[-2:]}
            # Find all h1s and h2s
            h1s = soup.find_all('h1')
            h2s = soup.find_all('h2')
            h1_texts = [h1.get_text(strip=True) for h1 in h1s]
            h2_texts = [h2.get_text(strip=True) for h2 in h2s]
            headers = {'headers': list(filter(None,h1_texts+h2_texts))}
            pages.append({**first_relevant, **two_longest,**headers})
        else:
            pages.append({'headers':['Page not available']})

    return pages

# Takes response_text instead of a URL since the request is required to determine bs4/sel
def bs4_scrape(website_url: str, response_text) -> dict[str,str|dict[str,str]]:
    print(f'BS4 scraping {website_url}')
    soup = BeautifulSoup(response_text,'html.parser')
    url_results = {}

    #Scrape nav
    nav_list = bs4_nav_scrape(website_url, soup)
    if not nav_list[0]: return 'BS4 Nav list unavailable'
    url_results['nav'] = nav_list
    print(f'{up.urlparse(website_url).netloc} bs4 naver',nav_list)

    # Scrape home page and if there, first page
    urls = [website_url]
    if nav_list[1] and validators.url(nav_list[1]):
        urls.append(nav_list[1])
    pages = bs4_pages_scrape(urls)
    home_page_obj = pages[0]
    first_page_obj = pages[1] if len(pages) > 1 else {'headers':['First page unavailable']}
    url_results['home_page'], url_results['first_page'] = home_page_obj, first_page_obj

    # Extract headers
    url_results['headers'] = home_page_obj['headers'] + first_page_obj['headers']

    return url_results

urler = 'https://www.forsalebyowner.com/'
response = requests.get(urler)
print(response.url)
souper = BeautifulSoup(response.content, 'html.parser')
bs4_scrape(urler, response.text)

https://www.forsalebyowner.com/
BS4 scraping https://www.forsalebyowner.com/


'BS4 Nav list unavailable'

#### Selenium

In [17]:
# Gathers first two relevant chunks of texts, two longest chunks of text and all h1s and h2s from every url in list then closes stealth driver fed in
def sel_pages_scrape(driver, urls: list[str]) -> dict:
    pages = []
    for url in urls:
        if url and validators.url(url):
            driver.get(url)    
            time.sleep(2)
            page_text = driver.find_element("xpath","/html/body").text
            # Split on any whitespace (\n and \t)
            page_array = re.split(r'[\n\t]+',page_text)
            # Extract the first two pieces of text with more than (7) words -> to be tested
            first_relevant = {'first_relevant': [i for i in page_array if word_count(i) > 7][:2]}
            # Two longest pieces of text on the page. Test if this produces relevant results
            two_longest = {'two_longest': sorted(page_array,key=len)[-2:]}
            h1s = driver.find_elements("xpath","//h1")
            h2s = driver.find_elements("xpath","//h2")
            h1_texts = [h1.text for h1 in h1s if h1]
            h2_texts = [h2.text for h2 in h2s if h2]
            headers = {'headers':list(filter(None,h1_texts+h2_texts))}
            pages.append({**first_relevant, **two_longest,**headers})
        else:
            pages.append({'headers':['First page unavailable']})
    driver.close()
    return pages

def sel_scrape(url: str) -> dict[str,str|dict[str,str]]:
    print(f'Selenium scraping {url}')
    url_results = {}

    #Scrape nav
    driver = webdriver.Chrome()
    driver.get(url)
    nav_list = sel_nav_scrape(driver)
    driver.close()
    url_results['nav'] = nav_list
    print(f'{up.urlparse(url).netloc} sel naver', nav_list)

    # Configure driver to be passed throughout
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    stealth_driver = webdriver.Chrome(options=options)
    stealth(stealth_driver,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True,
            )
    stealth_driver.set_window_size(1100, 720)
    # stealth_driver.get(url)
    
    # Scrape home and first pages (requires both of these to have urls).
    home_page_obj, first_page_obj = sel_pages_scrape(stealth_driver, [url, nav_list[1]])
    url_results['home_page'], url_results['first_page'] = home_page_obj, first_page_obj

    # Extract headers
    url_results['headers'] = home_page_obj['headers'] + first_page_obj['headers']

    # stealth_driver.close()
    
    return url_results

# https://www.appliedlearning.com
# https://sunstonepartners.com
# https://ecmins.com
# https://www.dentalxchange.com/
# https://pulseca.com/
# https://cessco.ca/
# sel_scrape('https://www.dentalxchange.com/')
# options = Options()

driver = webdriver.Chrome()
sel_pages_scrape(driver,['https://www.forsalebyowner.com/','https://www.forsalebyowner.com/sellyourhome/package'])

[{'first_relevant': ['Search our exclusive home inventory. Enter an address, neighborhood, or city',
   'From pricing your home to fielding offers, be in complete control of your home sale from your personalized dashboard.'],
  'two_longest': ['Taking steps, no matter how small, toward a sustainable lifestyle can feel incredibly gratifying – which is probably why...',
   'Selling a home yourself, without a big real estate agent’s network to help you, comes down to marketing your asset to bring in the best offers. You need to know your buyers and their needs.'],
  'headers': ['List Your Home With Confidence & Save',
   'Sell Your Home On Your Terms',
   'The Ultimate FSBO Seller Guide',
   'From pricing your home to fielding offers, be in complete control of your home sale from your personalized dashboard.',
   'Explore Top Marketplaces',
   'View Listings For Sale',
   'Take advantage of end to end support so you can spend more time on the things that matter.',
   'See Recently Sold & 

### Mistral exploration
Get HTML from a couple pages, see what GPT 3.5 gives. Mistral has 32k context window. \
**TODO:** SLIM REQUEST AND RESPONSE DOWN

In [ ]:
def query_mistral(website_object: str):
    # Point to the local server
    client = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")
    messages=[
            {"role": "system", "content": "As a master company assessment officer with 3000 years of experience, you have a task. Given an object with a data structure representing the navigation tree of a website and a handful of textual data like first relevant chunks of text and header values for the home page and first product page in the navigation tree, you have been instructed by the Prime Chancellor to tell me about the company and what your advice on investing in it would be. For example, information like the types of products that it sells and the sector that it is in."},
            # {"role": "system", "content": "{'nav': [[[[('Your Challenges', 'https://www.appliedlearning.com/your-challenges/'), [[('Overview', 'https://www.appliedlearning.com/your-challenges/')], [('Strategy Execution', 'https://www.appliedlearning.com/your-challenges/strategy-execution/'), [[('Overview', 'https://www.appliedlearning.com/your-challenges/strategy-execution/')], [('Achieve Strategy Alignment Within Executive Teams', 'https://www.appliedlearning.com/your-challenges/strategy-execution/achieve-strategy-alignment-within-executive-teams/')], [('Equip Managers to Turn Strategy into Action', 'https://www.appliedlearning.com/your-challenges/strategy-execution/equip-managers-to-turn-strategy-into-action/')], [('Gain Buy-in and Emotional Connection By Employees', 'https://www.appliedlearning.com/your-challenges/strategy-execution/gain-buy-in-and-emotional-connection-by-employees/')], [('Implement Strategy by Changing Key Processes', 'https://www.appliedlearning.com/your-challenges/strategy-execution/implement-strategy-by-changing-key-processes/')]]], [('Leadership Development', 'https://www.appliedlearning.com/your-challenges/leadership-development/')], [('Business Acumen', 'https://www.appliedlearning.com/your-challenges/business-acumen/')], [('Culture & Performance', 'https://www.appliedlearning.com/your-challenges/high-performance-culture/')], [('Operational Excellence', 'https://www.appliedlearning.com/your-challenges/operational-excellence/')]]], [('Our Approach', 'https://www.appliedlearning.com/authentic-employee-engagement/'), [[('Overview', 'https://www.appliedlearning.com/authentic-employee-engagement/')], [('Knowledge Maps', 'https://www.appliedlearning.com/authentic-employee-engagement/knowledge-maps/')], [('Visual Narratives', 'https://www.appliedlearning.com/authentic-employee-engagement/visual-narratives/')], [('Spark! Discussion Starters', 'https://www.appliedlearning.com/authentic-employee-engagement/discussion-starters/')], [('Now I See Workshops', 'https://www.appliedlearning.com/authentic-employee-engagement/now-i-see-workshops/')], [('eLearning', 'https://www.appliedlearning.com/authentic-employee-engagement/elearning/')]]], [('Clients', 'https://www.appliedlearning.com/clients/')], [('Insights', 'https://www.appliedlearning.com/insights/')], [('About', 'https://www.appliedlearning.com/about/')], [('Contact Us', 'https://www.appliedlearning.com/contact/')], [('', 'https://www.appliedlearning.com/#')]]], 'https://www.appliedlearning.com/your-challenges/'], 'home_page': {'first_relevant': ['Applied Learning has extensive experience addressing key business challenges. Our authentic employee engagement approach produces fresh solutions tailored to each clientâ€™s goals.', 'We believe that strategy execution requires leadership alignment as well as emotional buy-in at the employee level. We help align leadership teams and equip them to address the social side of strategy across your organization.'], 'two_longest': ['We provide expertise in knowing how to create authentic and engaging learning experiences that instill sound judgment at all levels of your organization. You can specify the program details or choose from proven business acumen frameworks that we offer.', 'We partner with you to develop custom solutions that deliver authentic employee engagement. Each expert-designed product described below is an effective stand-alone solution; when integrated together this blended solution will multiply and magnify results.'], 'headers': ['Applied Learning has extensive experience addressing key business challenges. Our authentic employee engagement approach produces fresh solutions tailored to each clientâ€™s goals.', 'Strategy Execution', 'Leadership Development', 'Business Acumen', 'Culture &\nPerformance', 'Operational Excellence', 'Authentic employee engagement is our people-centered approach to training and communication that integrates the needs of people, the possibilities of learning methodologies, and the business needs of our clients.']}, 'first_page': {'first_relevant': ['Applied Learning has extensive experience empowering organizations. Our authentic employee engagement approach will produce fresh solutions tailored to address your challenges and opportunities.', 'We believe that strategy execution requires leadership alignment as well as emotional buy-in at the employee level. We help align leadership teams and equip them to address the social side of strategy across your organization.'], 'two_longest': ['We have supported operational initiatives with hundreds of clients over the last 17 years. We offer expertise in creating tailored, experiential tools and programs that guide employees to personalize and act on operational initiatives.', 'We provide expertise in knowing how to create authentic and engaging learning experiences that instill sound judgment at all levels of your organization. You can specify the program details or choose from proven business acumen frameworks that we offer.'], 'headers': ['Applied Learning has extensive experience empowering organizations. Our authentic employee engagement approach will produce fresh solutions tailored to address your challenges and opportunities.', 'Strategy Execution', 'Leadership Development', 'Business Acumen', 'Culture &\nPerformance', 'Operational Excellence']}}"},
            # {"role": "user", "content": "['Accelerate your business and unlock new potential with the dental payments platform that connects your data, workflows, teams, and patients like never before.','Say goodbye to complexity. We address the full lifecycle of dental payments. From verifying eligibility to filing accurate claims and beyond, our powerful platform, intelligent data, and massive network ensure payers get exactly what they need so dentists get exactly what they’ve earned. Quickly, reliably, and easily.'],'two_longest': ['By integrating DentalXChange into your software, you can offer a more comprehensive product and give your users a seamless interface for submitting claims and tracking payments. And with our easy-to-use APIs, you’ve got quick access to trusted data and a vast payments network that can scale—with added value just around the corner.','You’re constantly seeking ways to improve your practice so you can focus on what you do best—dentistry. You need to get paid on time. And you want the payment process to be as smooth, seamless, and simple as possible. That’s where we come in. We help reduce your costs and stress—by streamlining your workflows and eliminating manual processes and paperwork. Submit claims correctly the first time. Know where you stand at any time. And create better experiences across the board for your patients.'"}
            {"role": "user", "content": "Here is the object containing information about the website: " + website_object + ". Please help me out and give me a well-informed output. Thank you."}
        ]
    while input != 'exit':

        completion = client.chat.completions.create(
        model="local-model", # this field is currently unused
        messages=messages,
        temperature=0.7,
        )

        new_message = {"role":"assistant","content":""}

        # type(completion) == <class 'openai.types.chat.chat_completion.ChatCompletion'>; type(chunk) == tuple
        for chunk in completion:
            print('chUNK:',chunk,type(chunk))
            if chunk[0] == 'choices':#.delta.content:
                new_message["content"] = chunk[1][0].message.content
                return new_message["content"]
                # TODO: Potentially continue the chat here but return repsonse for now. Also means change while loop to assess most recent message
                # messages.append(new_message)

        
        # print()
        # messages.append({"role":"user", "content": input("> ")})

objer = {'nav': [[[[('Your Challenges', 'https://www.appliedlearning.com/your-challenges/'), [[('Overview', 'https://www.appliedlearning.com/your-challenges/')], [('Strategy Execution', 'https://www.appliedlearning.com/your-challenges/strategy-execution/'), [[('Overview', 'https://www.appliedlearning.com/your-challenges/strategy-execution/')], [('Achieve Strategy Alignment Within Executive Teams', 'https://www.appliedlearning.com/your-challenges/strategy-execution/achieve-strategy-alignment-within-executive-teams/')], [('Equip Managers to Turn Strategy into Action', 'https://www.appliedlearning.com/your-challenges/strategy-execution/equip-managers-to-turn-strategy-into-action/')], [('Gain Buy-in and Emotional Connection By Employees', 'https://www.appliedlearning.com/your-challenges/strategy-execution/gain-buy-in-and-emotional-connection-by-employees/')], [('Implement Strategy by Changing Key Processes', 'https://www.appliedlearning.com/your-challenges/strategy-execution/implement-strategy-by-changing-key-processes/')]]], [('Leadership Development', 'https://www.appliedlearning.com/your-challenges/leadership-development/')], [('Business Acumen', 'https://www.appliedlearning.com/your-challenges/business-acumen/')], [('Culture & Performance', 'https://www.appliedlearning.com/your-challenges/high-performance-culture/')], [('Operational Excellence', 'https://www.appliedlearning.com/your-challenges/operational-excellence/')]]], [('Our Approach', 'https://www.appliedlearning.com/authentic-employee-engagement/'), [[('Overview', 'https://www.appliedlearning.com/authentic-employee-engagement/')], [('Knowledge Maps', 'https://www.appliedlearning.com/authentic-employee-engagement/knowledge-maps/')], [('Visual Narratives', 'https://www.appliedlearning.com/authentic-employee-engagement/visual-narratives/')], [('Spark! Discussion Starters', 'https://www.appliedlearning.com/authentic-employee-engagement/discussion-starters/')], [('Now I See Workshops', 'https://www.appliedlearning.com/authentic-employee-engagement/now-i-see-workshops/')], [('eLearning', 'https://www.appliedlearning.com/authentic-employee-engagement/elearning/')]]], [('Clients', 'https://www.appliedlearning.com/clients/')], [('Insights', 'https://www.appliedlearning.com/insights/')], [('About', 'https://www.appliedlearning.com/about/')], [('Contact Us', 'https://www.appliedlearning.com/contact/')], [('', 'https://www.appliedlearning.com/#')]]], 'https://www.appliedlearning.com/your-challenges/'], 'home_page': {'first_relevant': ['Applied Learning has extensive experience addressing key business challenges. Our authentic employee engagement approach produces fresh solutions tailored to each clientâ€™s goals.', 'We believe that strategy execution requires leadership alignment as well as emotional buy-in at the employee level. We help align leadership teams and equip them to address the social side of strategy across your organization.'], 'two_longest': ['We provide expertise in knowing how to create authentic and engaging learning experiences that instill sound judgment at all levels of your organization. You can specify the program details or choose from proven business acumen frameworks that we offer.', 'We partner with you to develop custom solutions that deliver authentic employee engagement. Each expert-designed product described below is an effective stand-alone solution; when integrated together this blended solution will multiply and magnify results.'], 'headers': ['Applied Learning has extensive experience addressing key business challenges. Our authentic employee engagement approach produces fresh solutions tailored to each clientâ€™s goals.', 'Strategy Execution', 'Leadership Development', 'Business Acumen', 'Culture &\nPerformance', 'Operational Excellence', 'Authentic employee engagement is our people-centered approach to training and communication that integrates the needs of people, the possibilities of learning methodologies, and the business needs of our clients.']}, 'first_page': {'first_relevant': ['Applied Learning has extensive experience empowering organizations. Our authentic employee engagement approach will produce fresh solutions tailored to address your challenges and opportunities.', 'We believe that strategy execution requires leadership alignment as well as emotional buy-in at the employee level. We help align leadership teams and equip them to address the social side of strategy across your organization.'], 'two_longest': ['We have supported operational initiatives with hundreds of clients over the last 17 years. We offer expertise in creating tailored, experiential tools and programs that guide employees to personalize and act on operational initiatives.', 'We provide expertise in knowing how to create authentic and engaging learning experiences that instill sound judgment at all levels of your organization. You can specify the program details or choose from proven business acumen frameworks that we offer.'], 'headers': ['Applied Learning has extensive experience empowering organizations. Our authentic employee engagement approach will produce fresh solutions tailored to address your challenges and opportunities.', 'Strategy Execution', 'Leadership Development', 'Business Acumen', 'Culture &\nPerformance', 'Operational Excellence']}}
query_mistral(str(objer))


chUNK: ('id', 'chatcmpl-oln4hbaaaug0fdvq7eg37bl') <class 'tuple'>
chUNK: ('choices', [Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Based on the information provided, Applied Learning appears to be a company that specializes in addressing business challenges and providing authentic employee engagement solutions. Their approach focuses on strategy execution, leadership development, business acumen, culture and performance, and operational excellence. They also offer expertise in creating tailored, experiential tools and programs that guide employees to personalize and act on operational initiatives.\nGiven this information, I would advise investing in Applied Learning if you are looking for a company that can help your organization address key business challenges and empower your employees through authentic employee engagement solutions. Their expertise in strategy execution, leadership development, business acumen, culture and performance, and operational

'Based on the information provided, Applied Learning appears to be a company that specializes in addressing business challenges and providing authentic employee engagement solutions. Their approach focuses on strategy execution, leadership development, business acumen, culture and performance, and operational excellence. They also offer expertise in creating tailored, experiential tools and programs that guide employees to personalize and act on operational initiatives.\nGiven this information, I would advise investing in Applied Learning if you are looking for a company that can help your organization address key business challenges and empower your employees through authentic employee engagement solutions. Their expertise in strategy execution, leadership development, business acumen, culture and performance, and operational excellence suggests that they have a strong understanding of the factors that contribute to organizational success. Additionally, their ability to create tailore

#### Demonstration

### Loop through excel

**Working version of script (not asynchronous, doesn't include redirect capturing)**

For each url imported from the excel:
0. Construct an output object
1. Get then add meta tags with JsonLink API
2. Try requests.get(url) and get output object with bs4 scraping methods
3. If response is not good, scrape with Selenium
4. Potentially add some color with local instance

In [17]:
def integrated_main():    
    start_time = time.time()
    index_range = slice(105,110)

    file_path = './Website_Redirects_230919.csv'
    df = pd.read_csv(file_path, low_memory=False)

    # Handle all redirects asynchronously

    if 'Website' not in df.columns:
        print("The CSV file must have a 'Website' column containing the URLs.")
        return
    backup_urls = df['Website'][index_range].tolist() 
    redirect_urls = df.get('Website Redirect', pd.Series(dtype=str)).tolist()[index_range]
    
    scrapes = []

    for i, redirect_url in enumerate(redirect_urls):
        current_url, current_data = '', {}
        # Check if 'Website Redirect' column is already populated (with valid URL)
        # if redirect_url.split(' ')[-1] == 'Error':
        #     scrapes += [{'nav':'Broken link','headers': '', 'home_page':'','first_page':''}]
        #     print(f'failed to process {backup_urls[i]} (index: {i+index_range.start}) - {scrapes[-1]}\n')
        #     continue
        if redirect_url and validators.url(redirect_url):
            current_url = redirect_url
        # TODO: this is to retry link, I should have (misplaced?) faith in the Website Redirect script
        elif backup_urls[i] and validators.url(backup_urls[i]):
            current_url = backup_urls[i]
        else:
            # Both the redirect_url and the backup_url are either nonexistent or invalid
            scrapes += [{'nav':'Broken link','headers': '', 'home_page':'','first_page':''}]
            continue
        
        # Handle scraping asynchronously

        # minimum layer is JsonLink
        # current_data = get_meta_tags(current_url)
        
        # Try request, if don't receive 200 status, use selenium
        # Either way, merge the result with current_data
        # TODO: need to account for requests.get() timing out
        try:
            current_response = requests.get(current_url)
        except:
            scrapes += [{'nav':'Broken link','headers': '', 'home_page':'','first_page':''}]
            continue
        if current_response.status_code//100 == 2:
            #BeautifulSoup scraping
            bs4_scrape_result = bs4_scrape(current_url, current_response.text)
            if bs4_scrape_result == 'BS4 Nav list unavailable':
                current_data = current_data | sel_scrape(current_url)
            else:
                current_data = current_data | bs4_scrape_result
        else:
            # Selenium scrape
            current_data = current_data | sel_scrape(current_url)

        # TODO: try mistral here
        # Requires server loaded and started in LM Studio
        # current_data["AI scrape"] = query_mistral(current_data)

        scrapes += [current_data]
        print(f'processed {current_url} (index: {i+index_range.start}) - {scrapes[-1]}\n')

    print('Updating Excel now')
    
    update_scrape_results(file_path, scrapes, index_range)

    # Asynchronous infrastructure

    # # Run the asynchronous function using asyncio.run()
    # loop = asyncio.get_event_loop()
    # final_urls = loop.run_until_complete(process_urls(valid_urls))

    # # Update 'Website Redirect' column in the CSV file with final URLs
    # update_scrape_results(file_path, valid_urls, final_urls, index_range)

    print(f"'Website Redirect' column updated in {time.time()-start_time} seconds.")

integrated_main()

bs4 scraping http://lstjobs.com/
lstjobs.com bs4 naver [[[[('', 'http://lstjobs.com/')], [[[[[('Home', 'http://lstjobs.com/')], [('About Us', 'http://lstjobs.com/about-us/')], [('Employment', 'http://lstjobs.com/employment/')], [('Services', 'http://lstjobs.com/our-services/')], [('Credit Application', 'http://lstjobs.com/credit-application/')], [('Testimonials', 'http://lstjobs.com/what-our-clients-say/')], [('Gallery', 'http://lstjobs.com/gallery/')], [('Contact Us', 'http://lstjobs.com/contact-us/')]]]], [[[[[('Home', 'http://lstjobs.com/')], [('About Us', 'http://lstjobs.com/about-us/')], [('Employment', 'http://lstjobs.com/employment/')], [('Services', 'http://lstjobs.com/our-services/')], [('Credit Application', 'http://lstjobs.com/credit-application/')], [('Testimonials', 'http://lstjobs.com/what-our-clients-say/')], [('Gallery', 'http://lstjobs.com/gallery/')], [('Contact Us', 'http://lstjobs.com/contact-us/')]]]]]]]], 'http://lstjobs.com/']
processed http://lstjobs.com/ (index

KeyError: 'website_redirect'

#### Ancillary Functions

In [8]:
def initial_processing(url):
    if not url or url != url or pd.isna(url):
        return ''
    
    # Sanitize URL
    corrected_url = sanitize_url(url)
    return corrected_url

# Function to sanitize/correct URLs missing pieces
def sanitize_url(url):
    # Parse URL to correct any issues then reconstruct
    parsed_url = urlparse(url)

    if not parsed_url.scheme:
    # Assume http scheme
        corrected_url = 'http://'+parsed_url.netloc + parsed_url.path + parsed_url.params + parsed_url.query + parsed_url.fragment
    else:
        corrected_url = parsed_url.geturl()

    return corrected_url

async def check_url(session, url, semaphore):
    async with semaphore:
        try:
            async with session.head(url, allow_redirects=True, timeout=100) as response:
                return str(response.url) # Return final URL as string
        # Catch errors
        except asyncio.TimeoutError as te:
            return 'Timeout Error'
        except aiohttp.ClientError as ce:
            return 'Client Error'
        except ValueError as ve:
            return 'Value Error'
        
async def process_urls(urls, MAX_CONCURRENT_REQUESTS):
    print(f"processing {len(urls)} urls")
    semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)
    async with aiohttp.ClientSession() as session:
        tasks = [check_url(session, url, semaphore) for url in urls]
        results = await asyncio.gather(*tasks)
        return results
    
async def return_invalid_url_object(url):
    return {'website_redirect': url, 'nav': 'Invalid_URL','home_page':{},'first_page':{},'headers':[]}

def update_redirect_urls(file_path, index_range, redirect_urls):
    df = pd.read_csv(file_path, low_memory=False)
    new_list = list(df['Website Redirect'][:index_range.start]) + redirect_urls + list(df['Website Redirect'][index_range.stop:])
    df['Website Redirect'] = new_list
    df.to_csv(file_path, index=False)

def construct_df_col(df, col_name: str, scrape_col: list, index_range: slice, col_exists: bool):
    if col_exists:
        return list(df[col_name][:index_range.start]) + scrape_col + list(df[col_name][index_range.stop:])
    else:
        return ['']*index_range.start + scrape_col + ['']*(len(df)-index_range.stop)

def update_scrape_results(file_path: str, scrape_results: list[dict], index_range: slice):
    df = pd.read_csv(file_path, low_memory=False)
    # TODO: Add 'Metadata' here when ready
    for column in ['Website Redirect','Nav','Headers','Home Page','First Page']:
        isolated_col = [result['_'.join(column.lower().split(' '))] for result in scrape_results]
        # isolated_col = [print(result.keys()) for result in scrape_results]
        df[column] = construct_df_col(df, column, isolated_col, index_range, column in df)
    # if 'Nav' in df:
    #     df_edit = list(df['Website Scrape'][:index_range.start]) + scrape_results + list(df['Website Scrape'][index_range.stop:])
    #     print('DF lengths', len(df),len(df_edit),len(list(df['Website Scrape'][:index_range.start])),len(scrape_results),len(list(df['Website Scrape'][index_range.stop:])),index_range,df['Website Scrape'][:index_range.start], df['Website Scrape'][index_range.stop:], scrape_results)
    #     df['Website Scrape'] = df_edit #[index_range] = scrape_results
    # else:
    #     # print(len(df),len(scrape_results))
    #     scrape_results = ['']*index_range.start + scrape_results + ['']*(len(df)-index_range.stop)
    #     df['Website Scrape'] = scrape_results
    df.to_csv(file_path, index=False)

### Integrated Main

In [18]:
import asyncio
import aiohttp
import pandas as pd
from urllib.parse import urlparse
import validators
import time
from concurrent.futures import ThreadPoolExecutor

nest_asyncio.apply()

async def scrape_website(session, url, semaphore, executor):
    async with semaphore:
        try:
            # First, attempt to scrape using aiohttp (or requests)
            async with session.get(url, allow_redirects=True, timeout=15) as response:
                if response.status//100 == 2:
                    # response.text() coroutine for asynchroneity
                    response_text = await response.text()
                    bs4_result = bs4_scrape(url, response_text)
                    if bs4_result == 'BS4 Nav list unavailable':
                        raise Exception('BS4 doesn\'t know where to go')
                    # If bs4 finds the necessary data, return it
                    return bs4_result
                else:
                    raise Exception(f"Non-200 response -> {response.status}")
        except Exception as e:
            print(f'Error with {url}:',e)
            # Fallback to Selenium scraping within the thread pool executor
            loop = asyncio.get_event_loop()
            return await loop.run_in_executor(executor, lambda: sel_scrape(url))

async def main_async(input_urls):
    tasks = []
    scrape_tasks = []
    MAX_CONCURRENT_REQUESTS = 1000
    scrape_results = {}
    final_urls = []
    semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)
    executor = ThreadPoolExecutor(max_workers=10)
    async with aiohttp.ClientSession() as session:
        # Capture redirects
        for url in input_urls:
            task = asyncio.create_task(check_url(session, url, semaphore))
            tasks.append(task)
        redirect_urls = await asyncio.gather(*tasks)

        scrape_results['website_redirect'] = redirect_urls

        # return scrape_results
                
        # Construct list of final_urls
        for i, redirect_url in enumerate(redirect_urls):
            current_url, current_data = '', {}
            # Check if 'Website Redirect' column is already populated (with valid URL) if error, skip
            # if redirect_url.split(' ')[-1] == 'Error':
            #     scrapes += [{'nav':'Broken link','headers': '', 'home_page':'','first_page':''}]
            #     print(f'failed to process {backup_urls[i]} (index: {i+index_range.start}) - {scrapes[-1]}\n')
            #     continue
            if redirect_url and validators.url(redirect_url):
                current_url = redirect_url
            # TODO: this is to retry link, I should have (misplaced?) faith in the Website Redirect script
            elif input_urls[i]:
                if redirect_url in ['Client Error', 'Timeout Error', 'Value Error']:
                    current_url = 'Invalid_URL'
                elif validators.url(input_urls[i]):
                    current_url = input_urls[i]
            else:
                # Both the redirect_url and the backup_url are either nonexistent or invalid
                # TODO: Add this functionality back in through bs4/sel
                # scrapes += [{'nav':'Broken link','headers': '', 'home_page':'','first_page':''}]
                current_url = 'Invalid_URL'
                continue
            final_urls.append(current_url)

        print('finals',redirect_urls,final_urls)

        # Schedule scraping tasks for each final URL
        for url in final_urls:
            if validators.url(url):
                scrape_task = asyncio.create_task(scrape_website(session, url, semaphore, executor))
                scrape_tasks.append(scrape_task)
            else:
                scrape_task = asyncio.create_task(return_invalid_url_object(url))
                scrape_tasks.append(scrape_task)
        scrape_results = await asyncio.gather(*scrape_tasks)

        # Consolidate redirects and scrape results
        # scrape_results = await asyncio.gather(*scrape_tasks)

        # print('arrs',len(redirect_urls),redirect_urls,'SCRAPER',scrape_results, len(scrape_results))

        website_results = [{'website_redirect':redirect, **scrape_results[i]} for i, redirect in enumerate(redirect_urls)]

        # print('scres',scrape_results, redirect_urls)
        # print('wbres',website_results)

        # Return list of tuples with (Website_redirect, nav, headers, home_page, first_page)
        return website_results

def integrated_main2():
    start_time = time.time()
    # Excel index = 2 + this index
    start, stop = 116, 117
    if stop <= start:
        print('Start must be strictly less than stop')
        return 1
    index_range = slice(start, stop)

    # Load your URLs from a file or list
    file_path = './Website_Redirects_230919.csv'
    df = pd.read_csv(file_path, low_memory=False)
    raw_urls = df['Website'][index_range].tolist()
    redirect_urls = df.get('Website Redirect', pd.Series(dtype=str)).tolist()[index_range]

    # Check if 'Website Redirect' column is already populated (with valid URL)
    for i, redirect_url in enumerate(redirect_urls):
        if redirect_url and validators.url(redirect_url):
            raw_urls[i] = redirect_url

    # Process the URLs asynchronously
    sanitized_urls = [initial_processing(url) for url in raw_urls]
    valid_urls = [url if validators.url(url) else '' for url in sanitized_urls]

    # Run the asynchronous tasks
    loop = asyncio.get_event_loop()
    scrape_results = loop.run_until_complete(main_async(valid_urls))

    update_scrape_results(file_path, scrape_results, index_range)

    print(f"Completed in {time.time() - start_time} seconds. Excel updated.")

integrated_main2()

finals ['https://www.rootermancan.com'] ['https://www.rootermancan.com']
BS4 scraping https://www.rootermancan.com
www.rootermancan.com bs4 naver [[[('Open toolbar', '')], [[[[('Increase Text', 'https://www.rootermancan.com')], [('Decrease Text', 'https://www.rootermancan.com')], [('Grayscale', 'https://www.rootermancan.com')], [('High Contrast', 'https://www.rootermancan.com')], [('Negative Contrast', 'https://www.rootermancan.com')], [('Light Background', 'https://www.rootermancan.com')], [('Links Underline', 'https://www.rootermancan.com')], [('Readable Font', 'https://www.rootermancan.com')], [('Reset', 'https://www.rootermancan.com')]]]]], 'No href found']
Completed in 8.60122537612915 seconds. Excel updated.


In [143]:
requests.get('https://cessco.ca/')

<Response [403]>

In [104]:
urlh = 'https://www.themedicalmemory.com'

async with aiohttp.ClientSession() as session:
     async with session.get(urlh, allow_redirects=True, timeout=50) as response:

        # print('STATUS', response.status,type(response.text()),type(response.text),type(response))
        if response.status == 200:
            # bs4_soup = BeautifulSoup(response.text(), 'html.parser')
            # print('BS4 result',bs4_soup)
            # response.text() coroutine for asynchroneity
            response_text = await response.text()
            print('r',type(response_text), response_text)
            bs4_result = bs4_scrape(urlh, response_text)

        try:
            response = requests.get(urlh)
            # res = bs4_scrape(urlh,response.text)
            soup1 = BeautifulSoup(response.text, 'html.parser')
            # soup2 = BeautifulSoup(await response.text(), 'html.parser')
            bs5_result = bs4_scrape(urlh, response.text)
            # print(res)
        except Exception as e:
            print('fail', e)

print('res',bs4_result, bs5_result)

# sel_scrape('http://www.academicresourcesolutions.com')

r <class 'str'> <!DOCTYPE html>
<html lang="en">
<head>
  
  <meta charset='utf-8'>
  <meta name="viewport" content="width=device-width, initial-scale=1" id="wixDesktopViewport" />
  <meta http-equiv="X-UA-Compatible" content="IE=edge">
  <meta name="generator" content="Wix.com Website Builder"/>

  <link rel="icon" sizes="192x192" href="https://static.wixstatic.com/media/a8bf39_d3b787db52d4470d8d40e1cb866f9976%7Emv2.jpg/v1/fill/w_192%2Ch_192%2Clg_1%2Cusm_0.66_1.00_0.01/a8bf39_d3b787db52d4470d8d40e1cb866f9976%7Emv2.jpg" type="image/jpeg"/>
  <link rel="shortcut icon" href="https://static.wixstatic.com/media/a8bf39_d3b787db52d4470d8d40e1cb866f9976%7Emv2.jpg/v1/fill/w_32%2Ch_32%2Clg_1%2Cusm_0.66_1.00_0.01/a8bf39_d3b787db52d4470d8d40e1cb866f9976%7Emv2.jpg" type="image/jpeg"/>
  <link rel="apple-touch-icon" href="https://static.wixstatic.com/media/a8bf39_d3b787db52d4470d8d40e1cb866f9976%7Emv2.jpg/v1/fill/w_180%2Ch_180%2Clg_1%2Cusm_0.66_1.00_0.01/a8bf39_d3b787db52d4470d8d40e1cb866f9976%7Emv

In [63]:
a = {'b':3,'c': 4}
d = {'redirect':'asdf',**a}
d
requests.get('http://quickcarepharmacy.com')

<Response [404]>